In [15]:
import os
import requests
from typing import List, Tuple
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from newsapi import NewsApiClient
from openai import OpenAI

load_dotenv(verbose=True)
client = OpenAI(api_key=os.getenv("openaiAPI"))

NEWS_TOKEN = os.getenv("NewsAPI")
newsapi = NewsApiClient(api_key=NEWS_TOKEN)
top_headlines = newsapi.get_top_headlines(sources='bbc-news', page_size=1)

In [16]:
top_headlines

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Biden issues 39 presidential pardons and commutes 1,500 sentences',
   'description': 'The White House says it is the highest number of presidential acts of clemency in a single day.',
   'url': 'https://www.bbc.co.uk/news/articles/c30nq7r0v2mo',
   'urlToImage': 'https://ichef.bbci.co.uk/ace/branded_news/1200/cpsprodpb/660c/live/7ab59660-b875-11ef-a0f2-fd81ae5962f4.jpg',
   'publishedAt': '2024-12-12T12:52:16.6624166Z',
   'content': 'Announcing the move, Biden said those pardoned had "shown successful rehabilitation and have shown commitment to making their communities stronger and safer". Their non-violent convictions included d… [+2181 chars]'}]}

In [17]:
def get_article_from_url(url: str) -> str:
    """
    입력받은 URL을 토대로 기사의 본문 태그를 찾아 본문 내용을 반환합니다.
    """
    # get HTML content of url
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # select p tag to get content
    paragraphs = soup.find_all('p')
    article_text = '\n'.join([para.get_text() for para in paragraphs])
    
    return article_text

In [32]:
def convert_txt_to_steps(context: str, level: str):
    """
    입력받은 기사 내용(context)을 입력받은 수준(level)에 맞게 재생성합니다.
    """
    
    # set prompt for gpt model
    prompt = f"""
    I'll send you the article body. Please return the value in format like below.
    dict(“rewrite_article": string, “category": string, "keywords": list(str))

    1. Rewrite the article text to a {level} level and put it as the first part of the return value. You must finish article within 50 centences, and line it up appropriately to make it easier to read
    2. Select the category of t]orts, business, innovation, culture, travel, or earth, and enter it as the second value in the return value.
    3. Select 5-7 important words from the article in purpose of learning English and put them as the 3rd part of the return value

    Article Text: {context}
    """
    
    # get response form gpt model
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that rewrites articles for teaching English."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=2048,
        temperature=0.5
    )
    
    # return response text
    response_dict = response.model_dump()
    response_message = response_dict["choices"][0]["message"]["content"]
    return response_message


In [18]:
context = get_article_from_url(top_headlines["articles"][0]["url"])
context

https://www.bbc.co.uk/news/articles/c30nq7r0v2mo


'US President Joe Biden has issued presidential pardons to 39 Americans convicted of non-violent crimes, and commuted the sentences of nearly 1,500 others, including several convicted of multi-million dollar fraud schemes.\nThe White House described it as the most acts of presidential clemency issued in a single day. It comes after Biden made the decision to pardon his own criminally convicted son, Hunter Biden.\nAnnouncing the move, Biden said those pardoned had "shown successful rehabilitation and have shown commitment to making their communities stronger and safer". \nThe US Constitution decrees that a president has the broad "power to grant reprieves and pardons for offences against the United States, except in cases of impeachment".\nThe White House released a pardon list of 39 people that said the individuals had committed either a "non-violent offense" or a "non-violent drug offense". It did not name the specific crimes they were convicted of committing. \nSeveral of those pardo

In [29]:
leveled_text = convert_txt_to_steps(context, "elementary school")
leveled_text

'dict(\n    rewrite_article="""\nUS President Joe Biden has helped 39 people who were found guilty of non-violent crimes. \nHe also changed the sentences of almost 1,500 others, including some who were involved in big fraud cases. \nThis is the most pardons given by a president in one day. \nBiden decided to do this after he pardoned his own son, Hunter Biden, who had been found guilty of some crimes.\n\nBiden said that the people he pardoned have worked hard to improve their lives and help their communities. \nThe US Constitution allows the president to give pardons for crimes against the country, except when it comes to impeachment.\n\nThe list of 39 people who were pardoned included those who committed non-violent or drug-related crimes. \nThe specific crimes were not mentioned. \nMany of these people had already been released from prison. \nSome were veterans and had become leaders in their communities.\n\nOne person pardoned was a 49-year-old man from Virginia. \nHe was convicted 

In [31]:
eval(leveled_text)

{'rewrite_article': "\nUS President Joe Biden has helped 39 people who were found guilty of non-violent crimes. \nHe also changed the sentences of almost 1,500 others, including some who were involved in big fraud cases. \nThis is the most pardons given by a president in one day. \nBiden decided to do this after he pardoned his own son, Hunter Biden, who had been found guilty of some crimes.\n\nBiden said that the people he pardoned have worked hard to improve their lives and help their communities. \nThe US Constitution allows the president to give pardons for crimes against the country, except when it comes to impeachment.\n\nThe list of 39 people who were pardoned included those who committed non-violent or drug-related crimes. \nThe specific crimes were not mentioned. \nMany of these people had already been released from prison. \nSome were veterans and had become leaders in their communities.\n\nOne person pardoned was a 49-year-old man from Virginia. \nHe was convicted of a drug 

In [63]:
response = requests.post("https://news-ease.com/_/api/existed_word", params={"word": "check"})
print(response.text)
print(response.json().get("exist"))
print(type(response.json().get("exist")))

{"exist":false}
False
<class 'bool'>


In [62]:
response = requests.get("https://news-ease.com/_/api/max_word_id")
print(response.text)
print(response.json().get("max_word_id"))
print(type(response.json().get("max_word_id")))

{"max_word_id":-1}
-1
<class 'int'>


In [54]:
response.text

'An exception was raised. Check the application logs for details.'

In [36]:
leveled_text["rewrite_article"]

TypeError: string indices must be integers, not 'str'

In [ ]:
class ResponseDict:
    

In [33]:
changed_role_leveled_text = convert_txt_to_steps(context, "elementary school")
changed_role_leveled_text

'dict(\n    rewrite_article="""\n    President Joe Biden has helped 39 Americans who made mistakes in the past. \n    These people were convicted of non-violent crimes, which means they did not hurt anyone. \n    He also changed the sentences of almost 1,500 other people, including some who were involved in big fraud cases. \n    This is the most pardons given by a president in one day. \n    Biden decided to help these people after he pardoned his son, Hunter Biden, who had some legal troubles.\n\n    Biden said that the people he helped have worked hard to improve themselves and their communities. \n    The US Constitution allows a president to give pardons, which means forgiving someone for their crime. \n    The White House shared a list of the 39 people who were pardoned. \n    These people had either a non-violent crime or a drug-related crime. \n    However, the specific crimes were not shared.\n\n    Many of the pardoned individuals were released from prison. \n    Some were ve

In [73]:
test = str("""dict(rewrite_article="Daniel Burke, a former British soldier, was found dead in Ukraine after being killed by someone he knew. He had gone to Ukraine to help fight against Russia. The coroner, Zak Golombeck, said that Mr. Burke, who was 36 years old, died bravely but was killed in a cowardly way. The person accused of his death is an Australian man who has fled Ukraine and is wanted by the police there.\n\nDuring the inquest, it was revealed that Mr. Burke was shot while practicing drills far from the front lines of the war. The shooter, known as Abdelfetah 'Adam' Nourine, claimed he accidentally shot Mr. Burke. However, the coroner rejected this claim, stating that Mr. Burke was unarmed and could not defend himself.\n\nMr. Burke had previously fought in Syria against the Islamic State group after being deeply affected by the Manchester Arena bombing in 2017. He had formed a group called Dark Angels, which helped evacuate injured soldiers from the battlefield. \n\nOn August 11, the day he died, Mr. Burke spoke to his family and planned to contact them later that day. CCTV footage showed him with Mr. Nourine before they went to an abandoned training ground. After Mr. Burke was reported missing, police interviewed Mr. Nourine, who gave different accounts of what happened. He eventually led police to Mr. Burke's body, claiming it was an accident.\n\nA post-mortem examination revealed that Mr. Burke had been shot multiple times. Investigations indicated that it was unlikely the shooting was accidental, especially given the type of gun used. Ukrainian police have requested help from the Australian Consulate to question Mr. Nourine, but there has been no response yet.\n\nMr. Burke's mother expressed her heartbreak over her son's death, saying that she would never get over it. She believes that Mr. Nourine, as a trained soldier, should not have let the gun fire accidentally. The case remains under investigation, and there are calls for justice for Mr. Burke.","category":"culture","keywords":["soldier","Ukraine","inquest","accident","coroner","justice","military"])""")

In [75]:
test.find("category")

1978

In [81]:
test.find('category":')

1978

In [83]:
test = test.replace('category":', 'category"=')
print(test)

dict(rewrite_article="Daniel Burke, a former British soldier, was found dead in Ukraine after being killed by someone he knew. He had gone to Ukraine to help fight against Russia. The coroner, Zak Golombeck, said that Mr. Burke, who was 36 years old, died bravely but was killed in a cowardly way. The person accused of his death is an Australian man who has fled Ukraine and is wanted by the police there.

During the inquest, it was revealed that Mr. Burke was shot while practicing drills far from the front lines of the war. The shooter, known as Abdelfetah 'Adam' Nourine, claimed he accidentally shot Mr. Burke. However, the coroner rejected this claim, stating that Mr. Burke was unarmed and could not defend himself.

Mr. Burke had previously fought in Syria against the Islamic State group after being deeply affected by the Manchester Arena bombing in 2017. He had formed a group called Dark Angels, which helped evacuate injured soldiers from the battlefield. 

On August 11, the day he di

In [80]:
test.find('category:')

-1

In [100]:
import json

leveled_info = '{"rewrite_article": "On November 10, police believe Harshita Brella was strangled in Corby, Northamptonshire."}'
parsed_info = json.loads(leveled_info)
print(parsed_info)


{'rewrite_article': 'On November 10, police believe Harshita Brella was strangled in Corby, Northamptonshire.'}


In [101]:
parsed_info

{'rewrite_article': 'On November 10, police believe Harshita Brella was strangled in Corby, Northamptonshire.'}

In [94]:
test_list = "[ investigation, incident, arrested, fairground, safety, injuries, emergency ]"
test_list

'[ investigation, incident, arrested, fairground, safety, injuries, emergency ]'

In [99]:
test_list.replace(" ", "").replace("[", '["').replace("]", '"]').replace(",", '","')

'["investigation","incident","arrested","fairground","safety","injuries","emergency"]'

In [95]:
test_list_pc = test_list[test_list.find('[') + 1:]
test_list_pc

' investigation, incident, arrested, fairground, safety, injuries, emergency ]'

In [96]:
test_list_pc = test_list_pc[:test_list_pc.find(']')]
test_list_pc

' investigation, incident, arrested, fairground, safety, injuries, emergency '

In [97]:
test_list_pc.rstrip()

' investigation, incident, arrested, fairground, safety, injuries, emergency'

In [98]:
test_list_pc.lstrip()

'investigation, incident, arrested, fairground, safety, injuries, emergency '

In [ ]:
test_list_pc

In [78]:
test.find('keywords":')

1999

In [86]:
test

'dict(rewrite_article="Daniel Burke, a former British soldier, was found dead in Ukraine after being killed by someone he knew. He had gone to Ukraine to help fight against Russia. The coroner, Zak Golombeck, said that Mr. Burke, who was 36 years old, died bravely but was killed in a cowardly way. The person accused of his death is an Australian man who has fled Ukraine and is wanted by the police there.\n\nDuring the inquest, it was revealed that Mr. Burke was shot while practicing drills far from the front lines of the war. The shooter, known as Abdelfetah \'Adam\' Nourine, claimed he accidentally shot Mr. Burke. However, the coroner rejected this claim, stating that Mr. Burke was unarmed and could not defend himself.\n\nMr. Burke had previously fought in Syria against the Islamic State group after being deeply affected by the Manchester Arena bombing in 2017. He had formed a group called Dark Angels, which helped evacuate injured soldiers from the battlefield. \n\nOn August 11, the 

In [84]:
test = test.replace('keywords":', 'keywords"=')
print(test)

dict(rewrite_article="Daniel Burke, a former British soldier, was found dead in Ukraine after being killed by someone he knew. He had gone to Ukraine to help fight against Russia. The coroner, Zak Golombeck, said that Mr. Burke, who was 36 years old, died bravely but was killed in a cowardly way. The person accused of his death is an Australian man who has fled Ukraine and is wanted by the police there.

During the inquest, it was revealed that Mr. Burke was shot while practicing drills far from the front lines of the war. The shooter, known as Abdelfetah 'Adam' Nourine, claimed he accidentally shot Mr. Burke. However, the coroner rejected this claim, stating that Mr. Burke was unarmed and could not defend himself.

Mr. Burke had previously fought in Syria against the Islamic State group after being deeply affected by the Manchester Arena bombing in 2017. He had formed a group called Dark Angels, which helped evacuate injured soldiers from the battlefield. 

On August 11, the day he di

In [85]:
eval(test)

SyntaxError: unterminated string literal (detected at line 1) (<string>, line 1)

In [79]:
test.find('keywords:')

-1

In [37]:
evaled = eval(changed_role_leveled_text)
evaled

{'rewrite_article': "\n    President Joe Biden has helped 39 Americans who made mistakes in the past. \n    These people were convicted of non-violent crimes, which means they did not hurt anyone. \n    He also changed the sentences of almost 1,500 other people, including some who were involved in big fraud cases. \n    This is the most pardons given by a president in one day. \n    Biden decided to help these people after he pardoned his son, Hunter Biden, who had some legal troubles.\n\n    Biden said that the people he helped have worked hard to improve themselves and their communities. \n    The US Constitution allows a president to give pardons, which means forgiving someone for their crime. \n    The White House shared a list of the 39 people who were pardoned. \n    These people had either a non-violent crime or a drug-related crime. \n    However, the specific crimes were not shared.\n\n    Many of the pardoned individuals were released from prison. \n    Some were veterans who

In [64]:
"[" + ", ".join(evaled["key_words"]) + "]"

'[pardon, convicted, non-violent, community, sentence, fraud, rehabilitation]'

In [45]:
", ".join(evaled["key_words"])

'pardon, convicted, non-violent, community, sentence, fraud, rehabilitation'

In [38]:
evaled["rewrite_article"]

"\n    President Joe Biden has helped 39 Americans who made mistakes in the past. \n    These people were convicted of non-violent crimes, which means they did not hurt anyone. \n    He also changed the sentences of almost 1,500 other people, including some who were involved in big fraud cases. \n    This is the most pardons given by a president in one day. \n    Biden decided to help these people after he pardoned his son, Hunter Biden, who had some legal troubles.\n\n    Biden said that the people he helped have worked hard to improve themselves and their communities. \n    The US Constitution allows a president to give pardons, which means forgiving someone for their crime. \n    The White House shared a list of the 39 people who were pardoned. \n    These people had either a non-violent crime or a drug-related crime. \n    However, the specific crimes were not shared.\n\n    Many of the pardoned individuals were released from prison. \n    Some were veterans who served in the milit

In [39]:
evaled["category"]

'culture'

In [40]:
evaled["key_words"]

['pardon',
 'convicted',
 'non-violent',
 'community',
 'sentence',
 'fraud',
 'rehabilitation']

In [23]:
leveled_text

'dict(rewrite_article="President Joe Biden has decided to help 39 people who were found guilty of crimes that did not hurt anyone. He also shortened the sentences of almost 1,500 other people. This is the most pardons given in one day by a president. Biden\'s decision comes after he pardoned his son, Hunter Biden, who had some legal troubles. Biden said that the people he helped had worked hard to improve their lives and their communities. The president can grant pardons according to the US Constitution. The White House shared a list of the 39 people pardoned, but they did not say exactly what crimes they committed. Many of these people had already been released from prison. Some were veterans or leaders in their communities. One man from Virginia, who was pardoned, had made mistakes when he was younger but later graduated from college and served in the military. He is now known as a hardworking and trustworthy person. The 1,499 people who had their sentences shortened included some wh

In [30]:
first_output = "'dict(rewrite_article="President Joe Biden has decided to help 39 people who were found guilty of crimes that did not hurt anyone. He also shortened the sentences of almost 1,500 other people. This is the most pardons given in one day by a president. Biden\'s decision comes after he pardoned his son, Hunter Biden, who had some legal troubles. Biden said that the people he helped had worked hard to improve their lives and their communities. The president can grant pardons according to the US Constitution. The White House shared a list of the 39 people pardoned, but they did not say exactly what crimes they committed. Many of these people had already been released from prison. Some were veterans or leaders in their communities. One man from Virginia, who was pardoned, had made mistakes when he was younger but later graduated from college and served in the military. He is now known as a hardworking and trustworthy person. The 1,499 people who had their sentences shortened included some who were kept at home during the Covid-19 pandemic. Biden said these people deserved another chance. Among those who had their sentences shortened was a 76-year-old man who cheated many people out of their money. Another person was a former county commissioner who took bribes. Biden plans to take more actions in the future. He has given fewer pardons than many past presidents, but he has still helped some people in need. Earlier, he pardoned people convicted of simple marijuana possession and military personnel who were punished for their sexual orientation. Biden\'s decision to pardon his son has caused some disagreement because he had previously said he would not do it. Some people think the cases against Hunter were unfairly motivated. Recently, someone who used to work with the FBI admitted to lying about Biden and his son. This person made up stories about them accepting bribes. Former President Trump gave many pardons during his time in office and has promised to help people who were involved in the Capitol riot. He believes many of them should not be in jail. Biden\'s actions show he is trying to help those who have made mistakes but are trying to do better now.","category": "news","key_words": ["Biden", "pardons", "criminal", "rehabilitation", "sentences", "clemency", "community"])'"

SyntaxError: invalid syntax (3633545562.py, line 1)

In [24]:
response = eval(leveled_text)
response

SyntaxError: positional argument follows keyword argument (<string>, line 1)

### 테스트

In [2]:
NEWS_TOKEN = os.getenv("NewsAPI")
newsapi = NewsApiClient(api_key=NEWS_TOKEN)

In [9]:
top_headlines = newsapi.get_top_headlines(sources='bbc-news', page_size=1)
top_headlines

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': "Prince Harry 'one of two claimants' still suing Sun publisher",
   'description': 'Prince Harry and former Labour deputy leader Tom Watson are the two remaining claimants suing News Group Newspapers.',
   'url': 'https://www.bbc.co.uk/news/articles/cvglrv7q945o',
   'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/971d/live/f89a8890-a346-11ef-b0f5-15ab15c44ed7.jpg',
   'publishedAt': '2024-11-15T14:22:23.0305366Z',
   'content': 'The High Court in London saw both sides return, asking a judge to rule on preliminary issues before the trial in 2025.\r\nThese included whether Prince Harry could be allowed access to "relevant emails… [+1997 chars]'}]}

In [46]:
top_headlines = newsapi.get_top_headlines(q='bitcoin',
                                          sources='bbc-news')
top_headlines

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [43]:
n = 3
cnt = 0

bbc_cate_list = []
texts = newsapi.get_top_headlines(category='sports', language='en', page_size=100)
for text in texts["articles"]:
    if text["source"]["name"] == "BBC News":
        bbc_cate_list.append(text)
    
    if len(bbc_cate_list) == n:
        break

print(len(bbc_cate_list))
print(bbc_cate_list)

0
[]


In [29]:
top_headlines = newsapi.get_top_headlines(
    category='science',
    language='en',
    page_size=3
)
top_headlines


{'status': 'ok',
 'totalResults': 66,
 'articles': [{'source': {'id': None, 'name': 'Space.com'},
   'author': 'Brett Tingley',
   'title': "The final supermoon of 2024 rises tonight as November's Full Beaver Moon - Space.com",
   'description': 'November\'s Beaver Moon will appear in the sky next to the gorgeous Pleiades star cluster, also known as the "Seven Sisters."',
   'url': 'https://www.space.com/the-universe/moon/the-final-supermoon-of-2024-rises-nov-15-as-novembers-full-beaver-moon',
   'urlToImage': 'https://cdn.mos.cms.futurecdn.net/vsqQSPBUTUN9YhExZoytrc-1200-80.jpg',
   'publishedAt': '2024-11-15T12:40:53Z',
   'content': "The last of four consecutive supermoons in 2024 rises this week.\r\nNovember's Full Beaver Moon will rise on Friday (Nov. 15), just in time for those willing to stay up late for some weekend skywatchin… [+3361 chars]"},
  {'source': {'id': 'cnn', 'name': 'CNN'},
   'author': 'Jackie Wattles',
   'title': 'NASA worries space station leaks in Russian modu

In [ ]:
top

In [7]:
top_headlines

{'status': 'ok',
 'totalResults': 66,
 'articles': [{'source': {'id': None, 'name': 'Space.com'},
   'author': 'Brett Tingley',
   'title': "The final supermoon of 2024 rises tonight as November's Full Beaver Moon - Space.com",
   'description': 'November\'s Beaver Moon will appear in the sky next to the gorgeous Pleiades star cluster, also known as the "Seven Sisters."',
   'url': 'https://www.space.com/the-universe/moon/the-final-supermoon-of-2024-rises-nov-15-as-novembers-full-beaver-moon',
   'urlToImage': 'https://cdn.mos.cms.futurecdn.net/vsqQSPBUTUN9YhExZoytrc-1200-80.jpg',
   'publishedAt': '2024-11-15T12:40:53Z',
   'content': "The last of four consecutive supermoons in 2024 rises this week.\r\nNovember's Full Beaver Moon will rise on Friday (Nov. 15), just in time for those willing to stay up late for some weekend skywatchin… [+3361 chars]"},
  {'source': {'id': 'cnn', 'name': 'CNN'},
   'author': 'Jackie Wattles',
   'title': 'NASA worries space station leaks in Russian modu

In [1]:
import os
import requests
from typing import List, Tuple
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from newsapi import NewsApiClient
from openai import OpenAI

load_dotenv(verbose=True)
client = OpenAI(api_key=os.getenv("openaiAPI"))


def get_news_metainfo_from_bbc(n: int = 5) -> List[str]:
    """
    NewsApiClient를 사용하여 BBC의 그날 대표 뉴스 n개에 대한 메타 정보를 반환합니다.
    """
    # set tokens for NewsAPI
    NEWS_TOKEN = os.getenv("NewsAPI")
    newsapi = NewsApiClient(api_key=NEWS_TOKEN)
    
    # get top headlines of bbc-news
    top_headlines = newsapi.get_top_headlines(sources='bbc-news', page_size=n)
    
    # extract meta information from article lists
    articles = []
    for article in top_headlines['articles']:
        info_dict = dict()
        info_dict["title"] = article['title']
        info_dict["description"] = article['description']
        info_dict["url"] = article['url']
        info_dict["urlToImage"] = article['urlToImage']
        articles.append(info_dict)
        
    return articles
    
    
def get_article_from_url(url: str) -> str:
    """
    입력받은 URL을 토대로 기사의 본문 태그를 찾아 본문 내용을 반환합니다.
    """
    # get HTML content of url
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # select p tag to get content
    paragraphs = soup.find_all('p')
    article_text = '\n'.join([para.get_text() for para in paragraphs])
    
    return article_text


In [102]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.7 MB/s eta 0:00:00a 0:00:01


In [105]:
from nltk.corpus import wordnet
import nltk

nltk.download('wordnet')

word="happy"
synsets = wordnet.synsets(word)

for synset in synsets:
    print(f"Definition: {synset.definition()}")
    print(f"Synonyms: {[lemma.name() for lemma in synset.lemmas()]}")

[nltk_data] Downloading package wordnet to /Users/park-dong-
[nltk_data]     yeon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Definition: enjoying or showing or marked by joy or pleasure
Synonyms: ['happy']
Definition: marked by good fortune
Synonyms: ['felicitous', 'happy']
Definition: eagerly disposed to act or to be of service
Synonyms: ['glad', 'happy']
Definition: well expressed and to the point
Synonyms: ['happy', 'well-chosen']


In [111]:
(synsets[0]).definition()

'enjoying or showing or marked by joy or pleasure'

In [113]:
(synsets[0]).lemmas()

[Lemma('happy.a.01.happy')]

In [114]:
word="Korea"
synsets = wordnet.synsets(word)

for synset in synsets:
    print(f"Definition: {synset.definition()}")
    print(f"Synonyms: {[lemma.name() for lemma in synset.lemmas()]}")

Definition: an Asian peninsula (off Manchuria) separating the Yellow Sea and the Sea of Japan; the Korean name is Dae-Han-Min-Gook or Han-Gook
Synonyms: ['Korea', 'Korean_Peninsula', 'Dae-Han-Min-Gook', 'Han-Gook']


In [115]:
word="happy"
synsets = wordnet.synsets(word)

for synset in synsets:
    print(f"Definition: {synset.definition()}")
    print(f"Synonyms: {[lemma.name() for lemma in synset.lemmas()]}")

Definition: enjoying or showing or marked by joy or pleasure
Synonyms: ['happy']
Definition: marked by good fortune
Synonyms: ['felicitous', 'happy']
Definition: eagerly disposed to act or to be of service
Synonyms: ['glad', 'happy']
Definition: well expressed and to the point
Synonyms: ['happy', 'well-chosen']


In [124]:
word="happy"
synsets = wordnet.synsets(word)

In [126]:
print(len(synsets))

4


In [117]:
a = set()
b = [1, 2, 3, 1, 1]

a.update(b)
print(a)

{1, 2, 3}


In [128]:
meaning_value = ""
synonyms_value = ""

synonyms_list = set()
word="happy"

synsets = wordnet.synsets(word)

if len(synsets) != 0:    
    for idx, synset in enumerate(synsets):
        if idx == 0:
            meaning_value = synset.definition()
        synonyms_list.update([lemma.name() for lemma in synset.lemmas()])
    synonyms_list.discard(word) # delete original word
    synonyms_value = (", ".join(list(synonyms_list)))

print(meaning_value)
print(synonyms_value)

enjoying or showing or marked by joy or pleasure
well-chosen, glad, felicitous


In [123]:
synonyms_list.discard('happy')
print(synonyms_list)

{'well-chosen', 'glad', 'felicitous'}


In [122]:
synonyms_list

{'felicitous', 'glad', 'well-chosen'}